In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import re
import pandas as pd
import numpy as np
import time

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#Nasa News URL Setup
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)
time.sleep(2)

#html and soup
news_html = browser.html
news_soup = BeautifulSoup(news_html, 'html.parser')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\clair\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Getting NASA Mars News

In [3]:
#Getting NASA Mars News
title_head = news_soup.find_all('div', class_ = 'content_title')
title_head = title_head[1].text.strip()
print(f'Lastest Title: {title_head}')

Lastest Title: NASA Ingenuity Mars Helicopter Prepares for First Flight


In [4]:
para_head = news_soup.find_all('div', class_ = 'article_teaser_body')
para_head = para_head[0].text.strip()
print(f'Content: {para_head}')

Content: Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.


In [5]:
#browser.quit()

# Getting Space Images

In [17]:
#Setting Image URL
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [24]:
target_img = 'a[class="group  cursor-pointer block"]'
browser.find_by_tag(target_img).click()
image_html = browser.html
image_soup = BeautifulSoup(image_html, 'html.parser')

image_href = image_soup.find_all('div', class_ = 'lg:w-auto w-full')
featured_image_url = image_href[0].a['href']
featured_image_url
print(image_href[0].a['href'])



https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24510.jpg


# Mars Facts

In [8]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [10]:
facts = pd.read_html(facts_url)
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [11]:
facts_df = facts[0]
facts_df.columns = ['Description', 'Value']
facts_df.set_index('Description', inplace=True)
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Mars Hemispheres 

In [12]:
hemp_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemp_url)


In [13]:
hemp_html = browser.html
hemp_soup = BeautifulSoup(hemp_html, 'html.parser')
content = hemp_soup.find_all('div', class_='item')
content

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [14]:
hemp_title = []
hemp_url = []
for i in content:
    hemp_title.append(i.find('h3').text.strip())
    hemp_url.append('https://astrogeology.usgs.gov/' + i.find('a')['href'])
hemp_url
hemp_title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [15]:
img_full_url = []
for full in hemp_url:
    browser.visit(full)
    img_full_html = browser.html
    img_full_soup = BeautifulSoup(img_full_html, 'html.parser')

    full = 'https://astrogeology.usgs.gov' + img_full_soup.find('img',class_ ='wide-image')['src']
    img_full_url.append(full)
    
img_full_url

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [16]:
hemisphere_image_urls = []

for dictionary in range(len(img_full_url)):
    hemisphere_image_urls.append({'Title':hemp_title[dictionary],'Img_url':img_full_url[dictionary]})

hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]